In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne
from autoreject import get_rejection_threshold
from mne_icalabel import label_components
from numpy import save

%matplotlib qt

In [ ]:
project_url = 'C:/Users/iqbal/Desktop/CogWear/Dataset/survey_gamification/'

In [ ]:
participant_id = 23 # 0 - 10
id = str(participant_id)

In [ ]:
session_type = 'post'

In [ ]:
session = 'cognitive_load' # baseline / cognitive_load

In [ ]:
data_type = 'muse_eeg'

In [ ]:
data_url = project_url + id + '/' + session_type + '/' + session + '/' + data_type + '.csv'

In [ ]:
data = pd.read_csv(data_url, usecols = ['RAW_TP9', 'RAW_AF7', 'RAW_AF8', 'RAW_TP10'])

In [ ]:
data = data.dropna()

In [ ]:
#print('Session Duration: ' + str(data.shape[0] / 256) + ' s')

In [ ]:
ch_names = ['TP9', 'AF7', 'AF8', 'TP10']
sfreq = 256
info = mne.create_info(ch_names = ch_names, sfreq = sfreq, ch_types = 'eeg')

In [ ]:
raw = mne.io.RawArray(data.T, info)

In [ ]:
#raw.plot(scalings = 2e2)

In [ ]:
raw.set_montage('standard_1020')

In [ ]:
'''
fig = raw.compute_psd().plot()

channels = raw.ch_names
colors = ['red', 'green', 'blue', 'black'] * int(len(channels)/4)

fig_psd = fig.axes[0]
fig_head = fig.axes[1]
fig_head.get_children()[0].set_color(colors)
fig_head.get_children()[0].set_linewidth(2.0)

for nl, (chan, color) in enumerate(zip(channels, colors)):
    line = fig_psd.get_lines()[nl + 2]
    line.set_color(color)
    if nl < 4:
        fig_head.get_lines()[nl].set_color('green')

fig.show()
'''

In [ ]:
#raw.plot_sensors(show_names=True)

In [ ]:
'''
def calcSNRCV(signal):
    data = signal.get_data()
    TP9_SNR = np.where(data[0].std() == 0, 0, data[0].mean() / data[0].std())
    TP9_CV = np.where(data[0].mean() == 0, 0, (data[0].std() / data[0].mean()) * 100)
    print('TP9 SNR: ' + str(TP9_SNR))
    print('TP9 Coefficient of Variation: ' + str(TP9_CV))
    AF7_SNR = np.where(data[1].std() == 0, 0, data[1].mean() / data[1].std())
    AF7_CV = np.where(data[1].mean() == 0, 0, (data[1].std() / data[1].mean()) * 100)
    print('AF7 SNR: ' + str(AF7_SNR))
    print('AF7 Coefficient of Variation: ' + str(AF7_CV))
    AF8_SNR = np.where(data[2].std() == 0, 0, data[2].mean() / data[2].std())
    AF8_CV = np.where(data[2].mean() == 0, 0, (data[2].std() / data[2].mean()) * 100)
    print('AF8 SNR: ' + str(AF8_SNR))
    print('AF8 Coefficient of Variation: ' + str(AF8_CV))
    TP10_SNR = np.where(data[3].std() == 0, 0, data[3].mean() / data[3].std())
    TP10_CV = np.where(data[3].mean() == 0, 0, (data[3].std() / data[3].mean()) * 100)
    print('TP10 SNR: ' + str(TP10_SNR))
    print('TP10 Coefficient of Variation: ' + str(TP10_CV))
'''

In [ ]:
#calcSNRCV(raw)

In [ ]:
raw_notch = raw.copy().notch_filter(freqs = 50)

In [ ]:
#raw_notch.plot(scalings = 2e2)

In [ ]:
'''
fig = raw_notch.compute_psd().plot()

channels = raw.ch_names
colors = ['red', 'green', 'blue', 'black'] * int(len(channels)/4)

fig_psd = fig.axes[0]
fig_head = fig.axes[1]
fig_head.get_children()[0].set_color(colors)
fig_head.get_children()[0].set_linewidth(2.0)

for nl, (chan, color) in enumerate(zip(channels, colors)):
    line = fig_psd.get_lines()[nl + 2]
    line.set_color(color)
    if nl < 4:
        fig_head.get_lines()[nl].set_color('green')

fig.show()
'''

In [ ]:
#calcSNRCV(raw_notch)

In [ ]:
raw_iir_butterworth_2 = raw_notch.copy().filter(l_freq = 1.0, h_freq = 50.0, method = 'iir', iir_params = dict(order = 1, ftype = 'butter'), verbose = True)
#raw_fir = raw_notch.copy().filter(l_freq = 1.0, h_freq = 50.0)

In [ ]:
#calcSNRCV(raw_fir)

In [ ]:
#calcSNRCV(raw_iir_butterworth_2)

In [ ]:
#raw_filt = raw_iir_butterworth_2

In [ ]:
#raw_filt.plot(scalings = 2e2)

In [ ]:
'''
fig = raw_filt.compute_psd().plot()

channels = raw.ch_names
colors = ['red', 'green', 'blue', 'black'] * int(len(channels)/4)

fig_psd = fig.axes[0]
fig_head = fig.axes[1]
fig_head.get_children()[0].set_color(colors)
fig_head.get_children()[0].set_linewidth(2.0)

for nl, (chan, color) in enumerate(zip(channels, colors)):
    line = fig_psd.get_lines()[nl + 2]
    line.set_color(color)
    if nl < 4:
        fig_head.get_lines()[nl].set_color('green')

fig.show()
'''

In [ ]:
#calcSNRCV(raw_filt)

In [ ]:
events = mne.make_fixed_length_events(raw_iir_butterworth_2, duration = 1)

In [ ]:
#epochs = mne.Epochs(raw, events, proj=True, reject=reject, preload=True, baseline = (None, 0))
epochs = mne.Epochs(raw_iir_butterworth_2, events, preload=True, baseline = (None, 0))

In [ ]:
#epochs.plot(scalings = 1e2)

In [ ]:
'''
fig = epochs.compute_psd().plot()

channels = raw.ch_names
colors = ['red', 'green', 'blue', 'black'] * int(len(channels)/4)

fig_psd = fig.axes[0]
fig_head = fig.axes[1]
fig_head.get_children()[0].set_color(colors)
fig_head.get_children()[0].set_linewidth(2.0)

for nl, (chan, color) in enumerate(zip(channels, colors)):
    line = fig_psd.get_lines()[nl + 2]
    line.set_color(color)
    if nl < 4:
        fig_head.get_lines()[nl].set_color('green')

fig.show()
'''

In [ ]:
#calcSNRCV(epochs)

In [ ]:
reject = get_rejection_threshold(epochs)

In [ ]:
epochs.drop_bad(reject=reject)

In [ ]:
#epochs.plot_drop_log()

In [ ]:
#epochs.plot(scalings = 1e2)

In [ ]:
'''
fig = epochs.compute_psd().plot()

channels = raw.ch_names
colors = ['red', 'green', 'blue', 'black'] * int(len(channels)/4)

fig_psd = fig.axes[0]
fig_head = fig.axes[1]
fig_head.get_children()[0].set_color(colors)
fig_head.get_children()[0].set_linewidth(2.0)

for nl, (chan, color) in enumerate(zip(channels, colors)):
    line = fig_psd.get_lines()[nl + 2]
    line.set_color(color)
    if nl < 4:
        fig_head.get_lines()[nl].set_color('green')

fig.show()
'''

In [ ]:
#calcSNRCV(epochs)

In [ ]:
evoked = epochs.average()

In [ ]:
'''
fig = evoked.plot(gfp = True)

channels = evoked.ch_names
colors = ['red', 'green', 'blue', 'black'] * int(len(channels)/4)

fig_psd = fig.axes[0]
fig_head = fig.axes[1]
fig_head.get_children()[0].set_color(colors)
fig_head.get_children()[0].set_linewidth(2.0)

for nl, (chan, color) in enumerate(zip(channels, colors)):
    line = fig_psd.get_lines()[nl]
    line.set_color(color)
    if nl < 4:
        fig_head.get_lines()[nl].set_color('green')
'''

In [ ]:
#evoked.plot_joint(times = 'peaks')
#evoked.plot_topomap(times = 'peaks', show_names = True)

In [ ]:
random_state = 42
ica_n_components = .99
ica = mne.preprocessing.ICA(n_components = ica_n_components, random_state=random_state, method = 'fastica')

In [ ]:
ica.fit(epochs)

In [ ]:
#ica.plot_components()

In [ ]:
#ica.plot_properties(epochs, picks=range(0, ica.n_components_))

In [ ]:
#ica.plot_sources(epochs, show_scrollbars=False)

In [ ]:
#ica.plot_overlay(raw, exclude=[0])

In [ ]:
#ica.plot_overlay(raw, exclude=[1])

In [ ]:
#ica.plot_overlay(raw, exclude=[2])

In [ ]:
#ica.plot_overlay(raw, exclude=[3])

In [ ]:
label = label_components(epochs, ica, method='iclabel')

In [ ]:
print(label)

In [ ]:
components = []
for i in range(len(label['labels'])):
    if label['labels'][i] != 'brain':
        components.append(i)

In [ ]:
print(components)

In [ ]:
#ica.plot_overlay(raw, exclude=components)

In [ ]:
ica.exclude = components

In [ ]:
epochs_postica = ica.apply(epochs.copy())

In [ ]:
#epochs_postica.plot(scalings = 1e2)

In [ ]:
'''
fig = epochs_postica.compute_psd().plot()

channels = raw.ch_names
colors = ['red', 'green', 'blue', 'black'] * int(len(channels)/4)

fig_psd = fig.axes[0]
fig_head = fig.axes[1]
fig_head.get_children()[0].set_color(colors)
fig_head.get_children()[0].set_linewidth(2.0)

for nl, (chan, color) in enumerate(zip(channels, colors)):
    line = fig_psd.get_lines()[nl + 2]
    line.set_color(color)
    if nl < 4:
        fig_head.get_lines()[nl].set_color('green')

fig.show()
'''

In [ ]:
#calcSNRCV(epochs_postica)

In [ ]:
x = epochs_postica.get_data()

In [ ]:
x.shape

In [ ]:
X = np.swapaxes(x, 1, 2)

In [ ]:
X.shape

In [ ]:
y = np.full((X.shape[0]), session)

In [ ]:
y.shape

In [ ]:
save('C:/Users/iqbal/Desktop/CogWear/' + session_type + '_' + session + '_' + 'X' + '_' + id + '.npy', X)

In [ ]:
save('C:/Users/iqbal/Desktop/CogWear/' + session_type + '_' + session + '_' + 'Y' + '_' + id + '.npy', y)

In [ ]:
#df_ica = epochs_postica.to_data_frame()

In [ ]:
#df_ica = df_ica.drop(columns = ['time', 'condition', 'epoch'])

In [ ]:
#df_ica['Label'] = session

In [ ]:
#df_ica.to_csv('C:/Users/iqbal/Desktop/CogWear/' + session + '_' + id + '.csv', index=False)